In [2]:
from io import BytesIO
from locale import currency
from os import times
from select import select
import time
from PIL import ImageGrab, Image
import win32api
import win32con
import win32gui
import ddddocr
import cv2 as cv
import numpy as np
import win32ui

In [63]:
ocr = ddddocr.DdddOcr()
name = str(input('模拟器名称：\n'))
hwnd = win32gui.FindWindow(None,name)
hwnd = win32gui.FindWindowEx(hwnd,None,None,None)
x_1, y_1, x_2, y_2 = win32gui.GetWindowRect(hwnd)
width = x_2 - x_1
height = y_2 - y_1

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
模拟器名称：
ld


In [64]:
def cvRead(img_path):
    return cv.imdecode(np.fromfile(img_path, dtype=np.uint8), -1)

In [98]:
#鼠标按下
def mouseDown(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, press_pos)

#鼠标松开
def mouseUp(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONUP,win32con.MK_LBUTTON, press_pos)

#鼠标点击
def mouseClick(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, press_pos)
    time.sleep(0.2)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONUP,None, press_pos)
    
#鼠标双击
def doubleClick(x:int, y:int):
    mouseClick(x, y)
    time.sleep(0.1)
    mouseClick(x, y)

#向下滑动
def swipeDown():
    win32gui.SendMessage(hwnd, win32con.WM_KEYDOWN,win32con.VK_SPACE,0)
    time.sleep(1.2)
    win32gui.SendMessage(hwnd, win32con.WM_KEYUP,win32con.VK_SPACE,0)

def swipeLeft():
    win32gui.SendMessage(hwnd, win32con.WM_KEYDOWN,win32con.VK_CONTROL,0)
    time.sleep(1.1)
    win32gui.SendMessage(hwnd, win32con.WM_KEYUP,win32con.VK_CONTROL,0)


In [66]:
def capWindow():
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, width, height)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (width, height), dcObj, (0, 0), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = np.fromstring(bmpstr, dtype='uint8')
    img.shape = (height,width, 4)
    img = img[...,:3]
    img = np.ascontiguousarray(img)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())

    return img

In [67]:
def capWindowPos(x:int, y:int, x1:int, y1:int):
    w = x1 - x
    h = y1 - y
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h), dcObj, (x,y), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = np.fromstring(bmpstr, dtype='uint8')
    img.shape = (h, w, 4)
    img = img[...,:3]
    img = np.ascontiguousarray(img)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    return img

In [68]:
def OCR(x,y,x1,y1):
    w = x1 - x
    h = y1 - y
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h), dcObj, (x,y), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = Image.frombuffer(
        'RGB',
        (bmpinfo['bmWidth'], bmpinfo['bmHeight']),
        bmpstr, 'raw', 'BGRX', 0, 1)
    img_byte = BytesIO()  
    img.save(img_byte, format='PNG')
    img_info = img_byte.getvalue()
    result = ocr.classification(img_info)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())

    return result

In [109]:
def clickPic(tmp:str, threshold:float):
    scr = capWindow()
    tp = cvRead(tmp)
    result = cv.matchTemplate(scr, tp, cv.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
    if max_val < threshold:
        print(tmp, max_val, max_loc)
        return False
    #print(tmp, max_val, max_loc)
    mouseDown(int(max_loc[0]) + int(tp.shape[1]/2), int(max_loc[1]) + int(tp.shape[0]/2))
    time.sleep(0.5)
    mouseUp(int(max_loc[0]) + int(tp.shape[1]/2), int(max_loc[1]) + int(tp.shape[0]/2))
    
    return True

In [110]:
def findPic(x:int, y:int, x1:int, y1:int, tmp:str, threshold:float):
    scr = capWindowPos(x,y,x1,y1)
    tp = cvRead(tmp)
    result = cv.matchTemplate(scr, tp, cv.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
    if max_val < threshold:
        print(tmp, max_val, max_loc)
        return False
    print(tmp, max_val, max_loc)
    return True

In [111]:
def toHome():
    while 1:
        if OCR(17,244,63,268) != '调查':
            doubleClick(54,35)
            time.sleep(2)
        else: 
            time.sleep(1)
            break

In [112]:
def addCheck():
    while 1:
        if OCR(46,258,95,287) == '图鉴':
            a = OCR(1184,208,1208,226)  
            b = OCR(1184,299,1209,318)  
            c = OCR(1217,389,1243,408)  
            d = OCR(1217,480,1243,500)  
            e = OCR(1118,665,1173,686)  
            if e == '0':
                return 0
            if (a, b, c, d).count('0') == True:
                mouseClick(1211,187)  
                time.sleep(1.5)
                mouseClick(913,643)  
                time.sleep(1)
                mouseClick(1197,283)
                time.sleep(1)
                mouseClick(913,643)  
                time.sleep(1)
                if c == '0':
                    doubleClick(1194,403)  
                    time.sleep(1)
                    doubleClick(918,636)  
                    for i in range(4):
                        doubleClick(1001,463)
                    time.sleep(1)
                    doubleClick(1020,530)
                    time.sleep(1)
                    doubleClick(895,497)
                if d == '0':
                    doubleClick(1197,522)  
                    time.sleep(1)
                    doubleClick(918,636)  
                    for i in range(4):
                        doubleClick(1001,463)
                    time.sleep(1)
                    doubleClick(1020,530)
                    time.sleep(1)
                    doubleClick(895,497)
                doubleClick(54,35)
            if (a,b,c,d).count('0') == False:
                break

In [113]:
def pull():
    addCheck()
    time.sleep(1)
    doubleClick(1156,592)
    time.sleep(1.3)
    doubleClick(1156,592)

In [114]:
def drag():
    while 1:
        if findPic(466,109,634,151,'钓鱼\\蓝色.bmp',0.98):
            mouseDown(1156,592)
        elif findPic(740,98,811,170,'钓鱼\\绿色.bmp',0.99):
            mouseDown(1156,592)
        elif findPic(901,155,941,229,'钓鱼\\红色.bmp',0.97):
            mouseUp(1156,592)   
        if OCR(572,639,704,671) == '点击屏幕继续':
            time.sleep(0.5)
            doubleClick(639,651)
            time.sleep(3)
            doubleClick(639,651)
            time.sleep(3)
            doubleClick(948,605)  
            if OCR(543,127,602,161) == '来自':
                time.sleep(2)
                doubleClick(663,348)
                time.sleep(4)
                doubleClick(658,589)
            break
        if OCR(581,159,752,209) == '鱼儿跑了':
            time.sleep(2)
            break

In [115]:
def join(point):
    toHome()
    time.sleep(1.5)
    doubleClick(736,665)
    time.sleep(2)
    points = [
        '钓鱼\\1汇湾海滨.bmp',
        '钓鱼\\2湖湾公园.bmp',
        '钓鱼\\3白金海岸.bmp',
        '钓鱼\\4牧森湖.bmp',
        '钓鱼\\5.斯皮亚水库.bmp',
        '钓鱼\\6云霞群岛.bmp'
    ]
    result = findPic(480,48,1260,686,points[point],0.98)
    if result:
        clickPic(points[point],0.99)
    else:
        time.sleep(2)
        swipeDown()
        time.sleep(4)
        clickPic(points[point],0.99)

In [116]:
def fishing(times:int,point:int):
    _times = 0
    join(point)
    time.sleep(2)
    while _times < times:
        pull()
        time.sleep(2)
        drag()
        _times += 1

In [117]:
#a为钓鱼次数，b为钓鱼点====>0表示湖湾公园，5表示云霞群岛，只能选择0-5
"""修改a和b的值，然后运行"""
a = 5
b = 5

fishing(a, b)

钓鱼\6云霞群岛.bmp 0.9008362293243408 (0, 87)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728899002075195 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728899002075195 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.9447373747825623 (113, 4)
钓鱼\绿色.bmp 0.9728894829750061 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.9447373747825623 (113, 4)
钓鱼\绿色.bmp 0.9728894829750061 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.9447373747825623 (113, 4)
钓鱼\绿色.bmp 0.9728894829750061 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728896021842957 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728896021842957 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728894829750061 (22, 0)
钓鱼\红色.bmp 0.9643383026123047 (10, 9)
钓鱼\蓝色.bmp 0.944737434387207 (113, 4)
钓鱼\绿色.bmp 0.9728911519050598 (22

钓鱼\蓝色.bmp 0.9526312351226807 (75, 21)
钓鱼\绿色.bmp 0.9977201223373413 (44, 34)
完美按下
钓鱼\蓝色.bmp 0.9526312351226807 (75, 21)
钓鱼\绿色.bmp 0.9977201223373413 (44, 34)
完美按下
钓鱼\蓝色.bmp 0.9527060985565186 (99, 17)
钓鱼\绿色.bmp 0.9813362956047058 (9, 27)
钓鱼\红色.bmp 0.9609918594360352 (5, 15)
钓鱼\蓝色.bmp 0.9526943564414978 (98, 17)
钓鱼\绿色.bmp 0.9805483222007751 (8, 28)
钓鱼\红色.bmp 0.9609076380729675 (6, 16)
钓鱼\蓝色.bmp 0.9526943564414978 (98, 17)
钓鱼\绿色.bmp 0.9808843731880188 (25, 24)
钓鱼\红色.bmp 0.9611200094223022 (8, 16)
钓鱼\蓝色.bmp 0.9526617527008057 (77, 20)
钓鱼\绿色.bmp 0.9815412163734436 (23, 26)
钓鱼\红色.bmp 0.9613732099533081 (1, 32)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9820802807807922 (22, 29)
钓鱼\红色.bmp 0.9615335464477539 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9820802807807922 (22, 29)
钓鱼\红色.bmp 0.9617909789085388 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9811014533042908 (22, 29)
钓鱼\红色.bmp 0.9619954824447632 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0

钓鱼\蓝色.bmp 0.9559144377708435 (88, 18)
钓鱼\绿色.bmp 0.9810577630996704 (47, 34)
钓鱼\红色.bmp 0.9620295166969299 (9, 16)
钓鱼\蓝色.bmp 0.9560869932174683 (71, 21)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9618199467658997 (9, 16)
钓鱼\蓝色.bmp 0.9560869932174683 (71, 21)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9613574743270874 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9611915349960327 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804767966270447 (42, 33)
钓鱼\红色.bmp 0.9609968662261963 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9610230326652527 (1, 32)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9612408876419067 (1, 32)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804720282554626 (3, 26)
钓鱼\红色.bmp 0.9611961841583252 (1, 32)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.961387932300567

钓鱼\蓝色.bmp 0.9526766538619995 (77, 20)
钓鱼\绿色.bmp 0.9991133213043213 (48, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9984002709388733 (48, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9973067045211792 (48, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.99663907289505 (49, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9973241090774536 (49, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9980130195617676 (49, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9977281093597412 (49, 34)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9972044825553894 (49, 35)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9970870018005371 (49, 35)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9966058135032654 (49, 35)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9960848689079285 (49, 35)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9971444010734558 (50, 35)
完美按下
钓鱼\蓝色.bmp 0.9526767134666443 (

钓鱼\蓝色.bmp 0.9526766538619995 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9614225029945374 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9616758823394775 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9633671641349792 (7, 19)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9633671641349792 (7, 19)
钓鱼\蓝色.bmp 0.95267653465271 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9639983773231506 (1, 32)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9639983773231506 (1, 32)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9633520841598511 (9, 16)
钓鱼\蓝色.bmp 0.9526766538619995 (77, 20)
钓鱼\绿色.bmp 0.9804731011390686 (24, 29)
钓鱼\红色.bmp 0.9640702605247498 (9, 16)
钓鱼\蓝色.bmp 0.9526766538619995 (77, 20)
钓鱼\绿色.bmp 0.9804731011390686 (24, 29)
钓鱼\红色.bmp 0.9640702605247498 

...

钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804728031158447 (29, 30)
钓鱼\红色.bmp 0.9649500846862793 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804728031158447 (29, 30)
钓鱼\红色.bmp 0.9649500846862793 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9650546908378601 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804717898368835 (3, 26)
钓鱼\红色.bmp 0.9650923013687134 (9, 16)
钓鱼\蓝色.bmp 0.9526767730712891 (77, 20)
钓鱼\绿色.bmp 0.9804719090461731 (3, 26)
钓鱼\红色.bmp 0.9651938080787659 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651938080787659 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.965172290802002 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651491641998291 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9650778770446777

钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9808932542800903 (51, 35)
钓鱼\红色.bmp 0.9651983976364136 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9808932542800903 (51, 35)
钓鱼\红色.bmp 0.9651983976364136 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9824572801589966 (9, 43)
钓鱼\红色.bmp 0.9651814699172974 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9808356761932373 (40, 33)
钓鱼\红色.bmp 0.9651367664337158 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9808356761932373 (40, 33)
钓鱼\红色.bmp 0.9651367664337158 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.980823278427124 (42, 33)
钓鱼\红色.bmp 0.9650660157203674 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9838647842407227 (48, 34)
钓鱼\红色.bmp 0.9648936986923218 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9817479252815247 (49, 35)
钓鱼\红色.bmp 0.9646454453468323 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9817479252815247 (49, 35)
钓鱼\红色.bmp 0.9646454453

钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9648886322975159 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9650070071220398 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651098847389221 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651411175727844 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651983380317688 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9651983380317688 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9652749300003052 (9, 16)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9653455018997192 (8, 17)
钓鱼\蓝色.bmp 0.9526767134666443 (77, 20)
钓鱼\绿色.bmp 0.9804719686508179 (3, 26)
钓鱼\红色.bmp 0.9652634263038635 

钓鱼\绿色.bmp 0.9755377769470215 (12, 24)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9583192467689514 (19, 31)
钓鱼\绿色.bmp 0.975967526435852 (11, 24)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9663845300674438 (124, 28)
钓鱼\绿色.bmp 0.98213130235672 (4, 56)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9663845300674438 (124, 28)
钓鱼\绿色.bmp 0.98213130235672 (4, 56)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9703797101974487 (118, 15)
钓鱼\绿色.bmp 0.9823350310325623 (3, 58)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9703797101974487 (118, 15)
钓鱼\绿色.bmp 0.9823350310325623 (3, 58)
钓鱼\红色.bmp 0.9592400789260864 (0, 38)
钓鱼\蓝色.bmp 0.9712995886802673 (122, 27)
钓鱼\绿色.bmp 0.9832206964492798 (4, 56)
钓鱼\红色.bmp 0.9627780914306641 (2, 57)
钓鱼\蓝色.bmp 0.9715535640716553 (121, 24)
钓鱼\绿色.bmp 0.9833212494850159 (1, 58)
钓鱼\红色.bmp 0.9620304107666016 (19, 51)
钓鱼\蓝色.bmp 0.971812903881073 (119, 19)
钓鱼\绿色.bmp 0.9842413067817688 (7, 52)
钓鱼\红色.bmp 0.9593826532363892 (0, 50)
钓鱼\蓝色.bmp 0.97185450792312

钓鱼\蓝色.bmp 0.952897846698761 (68, 3)
钓鱼\绿色.bmp 0.9752760529518127 (4, 15)
钓鱼\红色.bmp 0.959419310092926 (0, 48)
钓鱼\蓝色.bmp 0.9532032012939453 (87, 27)
钓鱼\绿色.bmp 0.97552090883255 (4, 15)
钓鱼\红色.bmp 0.9594027400016785 (0, 48)
钓鱼\蓝色.bmp 0.9532032012939453 (87, 27)
钓鱼\绿色.bmp 0.97552090883255 (4, 15)
钓鱼\红色.bmp 0.9594027400016785 (0, 48)
钓鱼\蓝色.bmp 0.9537989497184753 (88, 29)
钓鱼\绿色.bmp 0.9755274057388306 (4, 15)
钓鱼\红色.bmp 0.959438681602478 (1, 46)
钓鱼\蓝色.bmp 0.9552505612373352 (128, 0)
钓鱼\绿色.bmp 0.9754033088684082 (4, 15)
钓鱼\红色.bmp 0.9594449996948242 (1, 46)
钓鱼\蓝色.bmp 0.954818069934845 (91, 31)
钓鱼\绿色.bmp 0.9753915667533875 (4, 15)
钓鱼\红色.bmp 0.9594376683235168 (1, 46)
钓鱼\蓝色.bmp 0.954879879951477 (91, 31)
钓鱼\绿色.bmp 0.9752899408340454 (4, 15)
钓鱼\红色.bmp 0.9594197273254395 (1, 46)
钓鱼\蓝色.bmp 0.9550479650497437 (91, 31)
钓鱼\绿色.bmp 0.9751318097114563 (4, 15)
钓鱼\红色.bmp 0.9593736529350281 (1, 47)
钓鱼\蓝色.bmp 0.9549778699874878 (91, 31)
钓鱼\绿色.bmp 0.9751229882240295 (4, 16)
钓鱼\红色.bmp 0.959347128868103 (1, 47)
钓鱼\